In [1]:
import RAP_MASTER_LIB
#MY LIBRARIES -- towards the end of project/when you need a break write up documentaiton of libraries
#another library for plotting and testing
from rap_scrpr import *
from rap_db import *
from rap_clean import*
from rap_viz import *

estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#import nltk
#nltk.download()
#create_music_tables(estconn)

import RAP_MASTER_LIB
import re
from bs4 import BeautifulSoup
import urllib
import time
import json
import psycopg2 as pg2
import psycopg2.extras

from rap_scrpr import *
from rap_db import *
from rap_clean import*
#nltk.download()
from os import listdir
from os.path import isfile, join

estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

In [2]:
#new_artists = scrape_multi_artists(estconn, ['10sion'])
#create_music_tables(estconn, bypass = True)
#bulk_load(estconn, new_artists)
#lst = ['Del', 'Eminem', 'Jay-Z']
#works_pull = dict(zip(lst, construct_artists(estconn, art_list = lst, use_ind_artists=False)))
#art_save(works_pull)

In [3]:
SequenceMatcher(None, "It could be fatal wait'll Del come back out \n and act out emotions in Oakland Blackouts", "Don't you ever forget, it could be fatal\nWait'll Del come back out, and act out emotions in Oakland Blackouts").ratio()

0.8367346938775511

In [4]:
works = art_load(['Doom', 'Del', 'Sweatshirt'])

In [5]:
unique_verses_bar(works.values())

In [6]:
unique_words_hist(works.values())

what you did
made viz library for organizing purposes
touched up viz functions
made pickle loading so we wouldn't have to make art objects everytime
also allows me to work on this on work pc

what's next
scatter plot on the mind
keep modeling and coming up with stuff!

In [7]:
#estconn.close()